In [1]:
import tensorflow as tf

import tensorflow.contrib.slim as slim
from mnist import inputs, lenet

In [2]:
flags = tf.app.flags
flags.DEFINE_string('train_dir', './data',
                    'Directory with the training data.')
flags.DEFINE_integer('batch_size', 5, 'Batch size.')
flags.DEFINE_integer('num_batches', 1000, 'Num of batches to evaluate.')
flags.DEFINE_string('log_dir', './log/eval',
                    'Directory where to log data.')
flags.DEFINE_string('checkpoint_dir', './log/train',
                    'Directory with the model checkpoint data.')
FLAGS = flags.FLAGS

In [3]:
images, labels = inputs(FLAGS.train_dir, False, FLAGS.batch_size, FLAGS.num_batches,one_hot_labels=False)
predictions = lenet(images)

image:  Tensor("input/Reshape:0", shape=(60, 80, 3), dtype=float32)
label1:  Tensor("input/Reshape_1:0", shape=(1,), dtype=float32)
label2:  Tensor("input/Reshape_1:0", shape=(1,), dtype=float32)
images
Tensor("input/shuffle_batch:0", shape=(5, 60, 80, 3), dtype=float32)


In [4]:
metrics_to_values, metrics_to_updates = slim.metrics.aggregate_metric_map({
    "streaming_mse": slim.metrics.streaming_mean_squared_error(predictions, labels),
})

In [5]:
metrics_to_values

{'streaming_mse': <tf.Tensor 'mean_squared_error/value:0' shape=() dtype=float32>}

In [6]:
metrics_to_updates

{'streaming_mse': <tf.Tensor 'mean_squared_error/update_op:0' shape=() dtype=float32>}

In [8]:
# for metric_name, metric_value in metrics_to_values.items():
tf.scalar_summary("metric_name", metrics_to_values['streaming_mse'])

<tf.Tensor 'ScalarSummary:0' shape=() dtype=string>

In [ ]:
slim.evaluation.evaluation_loop(
    '',
    FLAGS.checkpoint_dir,
    FLAGS.log_dir,
    num_evals=FLAGS.num_batches,
    eval_op=metrics_to_updates['streaming_mse'],
    summary_op=tf.merge_all_summaries(),
    eval_interval_secs=30)

In [21]:
# predictions = tf.to_int32(tf.argmax(predictions, 1))

# tf.scalar_summary('accuracy', slim.metrics.streaming_mean_squared_error(predictions, labels))

# These are streaming metrics which compute the "running" metric,
# e.g running accuracy
# metrics_to_values, metrics_to_updates = slim.metrics.aggregate_metric_map({
#     "streaming_mse": slim.metrics.streaming_mean_squared_error(predictions, labels),
# })

# Define the streaming summaries to write:
# for metric_name, metric_value in metrics_to_values.items():
#     tf.scalar_summary(metric_name, metric_value)
    
    
names_to_values, names_to_updates = slim.metrics.aggregate_metric_map({
  "mse": slim.metrics.streaming_mean_squared_error(predictions, labels),
})
# Define the summaries to write:
for metric_name, metric_value in metrics_to_values.items():
    tf.scalar_summary(metric_name, metric_value)
checkpoint_dir = './log/train'
log_dir = './log/eval'

RuntimeError: Graph is finalized and cannot be modified.